In [1]:
import tkinter as tk
from tkinter import filedialog, Scale, HORIZONTAL, Label, Button, Frame
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageTk


denoising_model = tf.keras.models.load_model('denoising_model.h5', custom_objects={'MeanSquaredError': tf.keras.losses.MeanSquaredError})
style_transfer_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')


img = None  # Current image being manipulated
original_img = None  # To store the original image for restoration
current_angle = 0  # For image rotation

def load_image():
    global img, original_img
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if file_path:
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for displaying
        original_img = img.copy()  # Keep a copy of the original image
        show_image(img)

def show_image(cv_img):
    cv_img = cv2.resize(cv_img, (800, 600))  # Resize for display
    pil_img = Image.fromarray(cv_img)
    img_display = ImageTk.PhotoImage(pil_img)
    image_label.config(image=img_display)
    image_label.image = img_display

def denoise_image():
    global img
    if img is not None:
        img_resized = cv2.resize(img, (256, 256))  # Resize for model input
        img_resized = img_resized / 255.0  # Normalize
        denoised_img = denoising_model.predict(np.expand_dims(img_resized, axis=0))[0]
        denoised_img = (denoised_img * 255).astype(np.uint8)  # Convert back to uint8
        show_image(denoised_img)

def load_style_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if file_path:
        apply_style_transfer(file_path)  # Pass the path of the style image

def apply_style_transfer(style_image_path):
    global img
    if img is not None:
        # Load and preprocess the style image
        style_image = load_and_preprocess_image(style_image_path)
        
        # Load and preprocess the content image from the global img variable
        content_image = load_and_preprocess_image_from_numpy(img)

        # Perform style transfer
        try:
            stylized_image = style_transfer_model(content_image, style_image)[0]
            stylized_image = np.squeeze(stylized_image.numpy())  # Remove batch dimension
            show_image((stylized_image * 255).astype(np.uint8))  # Convert back to uint8
        except Exception as e:
            print("Error during style transfer:", e)

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (512, 512))  # Resize to the required size for the model
    img = img / 255.0  # Normalize to [0, 1]
    img = img[tf.newaxis, :]  # Add batch dimension
    return img

def load_and_preprocess_image_from_numpy(image):
    img_resized = cv2.resize(image, (512, 512))
    img_resized = img_resized / 255.0  # Normalize to [0, 1]
    img_resized = img_resized[tf.newaxis, :]  # Add batch dimension
    return img_resized

# Image Manipulation Functions
def invert_colors():
    global img
    if img is not None:
        img = cv2.bitwise_not(img)
        show_image(img)

def convert_to_bw():
    global img
    if img is not None:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)  # Convert back to 3-channel
        show_image(img)

def adjust_brightness(val):
    global img
    if img is not None:
        brightness = int(val)
        img_adjusted = cv2.convertScaleAbs(original_img, alpha=1, beta=brightness)  # Use original for adjustments
        show_image(img_adjusted)

def adjust_contrast(val):
    global img
    if img is not None:
        contrast = float(val) / 10.0
        img_adjusted = cv2.convertScaleAbs(original_img, alpha=contrast, beta=0)  # Use original for adjustments
        show_image(img_adjusted)

def adjust_saturation(val):
    global img
    if img is not None:
        hsv = cv2.cvtColor(original_img, cv2.COLOR_RGB2HSV)  # Use original for adjustments
        hsv[..., 1] = hsv[..., 1] * (float(val) / 100.0)
        img_adjusted = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        show_image(img_adjusted)

def rotate_image(direction):
    global img, current_angle
    if img is not None:
        if direction == 'left':
            current_angle -= 45
        elif direction == 'right':
            current_angle += 45

        height, width = img.shape[:2]
        center = (width // 2, height // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, current_angle, 1)

        # Compute the bounding box of the new image after rotation
        abs_cos = abs(rotation_matrix[0, 0])
        abs_sin = abs(rotation_matrix[0, 1])
        new_width = int(height * abs_sin + width * abs_cos)
        new_height = int(height * abs_cos + width * abs_sin)

        # Adjust the rotation matrix to take the new dimensions into account
        rotation_matrix[0, 2] += (new_width / 2) - center[0]
        rotation_matrix[1, 2] += (new_height / 2) - center[1]

        rotated_img = cv2.warpAffine(img, rotation_matrix, (new_width, new_height))
        show_image(rotated_img)

def crop_image():
    global img, original_img
    if img is not None:
        height, width = original_img.shape[:2]
        
        top = int(crop_top.get() / 100 * height)
        bottom = int((100 - crop_bottom.get()) / 100 * height)
        left = int(crop_left.get() / 100 * width)
        right = int((100 - crop_right.get()) / 100 * width)

        cropped_img = original_img[top:bottom, left:right]  # Use original image for cropping
        show_image(cropped_img)

def edge_detection():
    global img
    if img is not None:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(img_gray, 100, 200)
        img_edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)  # Convert back to 3-channel
        show_image(img_edges)

def sharpen_image():
    global img
    if img is not None:
        kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
        img_sharpened = cv2.filter2D(img, -1, kernel)
        show_image(img_sharpened)

def smooth_image():
    global img
    if img is not None:
        img_smoothed = cv2.GaussianBlur(img, (5, 5), 0)
        show_image(img_smoothed)

# Highlight and Shadow Segmentation Functions
def segment_highlights():
    global img
    if img is not None:
        img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        # Define the range for highlights (e.g., high-value hues)
        lower_highlight = np.array([0, 0, 200])
        upper_highlight = np.array([180, 255, 255])
        mask_highlights = cv2.inRange(img_hsv, lower_highlight, upper_highlight)
        segmented_highlights = cv2.bitwise_and(img, img, mask=mask_highlights)
        segmented_highlights[np.where(mask_highlights == 0)] = 0  # Black out other areas
        show_image(segmented_highlights)

def segment_shadows():
    global img
    if img is not None:
        img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        # Define the range for shadows (e.g., low-value hues)
        lower_shadow = np.array([0, 0, 0])
        upper_shadow = np.array([180, 255, 50])
        mask_shadows = cv2.inRange(img_hsv, lower_shadow, upper_shadow)
        segmented_shadows = cv2.bitwise_and(img, img, mask=mask_shadows)
        segmented_shadows[np.where(mask_shadows == 0)] = 0  # Black out other areas
        show_image(segmented_shadows)

def reset_image():
    global img, original_img, current_angle
    img = original_img.copy()  # Restore the original image
    current_angle = 0  # Reset rotation
    show_image(img)  # Show the original image
    # Reset all sliders to their initial values
    brightness_slider.set(0)
    contrast_slider.set(10)
    saturation_slider.set(100)
    crop_top.set(0)
    crop_bottom.set(0)
    crop_left.set(0)
    crop_right.set(0)

# Initialize Tkinter window
root = tk.Tk()
root.title("Image Processing Tool")
root.geometry("1000x700")

# Label to display image
image_label = Label(root)
image_label.pack()

# Create a frame for buttons
button_frame = Frame(root)
button_frame.pack(side="top", pady=10)

# Buttons for controls
load_button = Button(button_frame, text="Load Image", command=load_image)
load_button.grid(row=0, column=0, padx=5)

denoise_button = Button(button_frame, text="Denoise Image", command=denoise_image, bg="green", fg="white")  # Green button
denoise_button.grid(row=0, column=1, padx=5)

style_button = Button(button_frame, text="Load Style Image", command=load_style_image, bg="green", fg="white")  # Green button
style_button.grid(row=0, column=2, padx=5)

invert_button = Button(button_frame, text="Invert Colors", command=invert_colors)
invert_button.grid(row=0, column=3, padx=5)

bw_button = Button(button_frame, text="Black & White", command=convert_to_bw)
bw_button.grid(row=0, column=4, padx=5)

highlight_button = Button(button_frame, text="Segment Highlights", command=segment_highlights)
highlight_button.grid(row=1, column=0, padx=5)

shadow_button = Button(button_frame, text="Segment Shadows", command=segment_shadows)
shadow_button.grid(row=1, column=1, padx=5)

edge_button = Button(button_frame, text="Edge Detect", command=edge_detection)
edge_button.grid(row=1, column=2, padx=5)

sharpen_button = Button(button_frame, text="Sharpen", command=sharpen_image)
sharpen_button.grid(row=1, column=3, padx=5)

smooth_button = Button(button_frame, text="Smooth", command=smooth_image)
smooth_button.grid(row=1, column=4, padx=5)

reset_button = Button(button_frame, text="Reset", command=reset_image, bg="red", fg="white", font=("Arial", 12, "bold"))
reset_button.grid(row=2, column=2, padx=5, pady=10)  # Prominent Reset Button

# Dynamic sliders
adjustment_frame = Frame(root)
adjustment_frame.pack(side="top", pady=10)

# Brightness slider
brightness_slider = Scale(adjustment_frame, from_=-100, to=100, orient=HORIZONTAL, label="Brightness", command=adjust_brightness)
brightness_slider.set(0)  # Start from 0
brightness_slider.grid(row=0, column=0, padx=10)

# Contrast slider
contrast_slider = Scale(adjustment_frame, from_=10, to=30, orient=HORIZONTAL, label="Contrast", command=adjust_contrast)
contrast_slider.set(10)  # Start from 10
contrast_slider.grid(row=0, column=1, padx=10)

# Saturation slider
saturation_slider = Scale(adjustment_frame, from_=0, to=200, orient=HORIZONTAL, label="Saturation", command=adjust_saturation)
saturation_slider.set(100)  # Start from 100
saturation_slider.grid(row=0, column=2, padx=10)

# Create cropping sliders
crop_frame = Frame(root)
crop_frame.pack(side="top", pady=10)

# Cropping sliders
crop_top = Scale(crop_frame, from_=0, to=100, orient=HORIZONTAL, label="Crop Top (%)", command=lambda x: crop_image())
crop_top.set(0)  # Start from 0
crop_top.grid(row=0, column=0, padx=5)

crop_bottom = Scale(crop_frame, from_=0, to=100, orient=HORIZONTAL, label="Crop Bottom (%)", command=lambda x: crop_image())
crop_bottom.set(0)  # Start from 0
crop_bottom.grid(row=0, column=1, padx=5)

crop_left = Scale(crop_frame, from_=0, to=100, orient=HORIZONTAL, label="Crop Left (%)", command=lambda x: crop_image())
crop_left.set(0)  # Start from 0
crop_left.grid(row=0, column=2, padx=5)

crop_right = Scale(crop_frame, from_=0, to=100, orient=HORIZONTAL, label="Crop Right (%)", command=lambda x: crop_image())
crop_right.set(0)  # Start from 0
crop_right.grid(row=0, column=3, padx=5)

# Rotation buttons
rotate_left_button = Button(button_frame, text="Rotate Left 45°", command=lambda: rotate_image('left'))
rotate_left_button.grid(row=2, column=0, padx=5)

rotate_right_button = Button(button_frame, text="Rotate Right 45°", command=lambda: rotate_image('right'))
rotate_right_button.grid(row=2, column=1, padx=5)

# Start the GUI loop
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
